In [2]:
%load_ext autoreload
%autoreload 2
# import os
# os.environ['CUDA_LAUNCH_BLOCKING'] = "1"
import argparse
import collections
import torch
import torch_geometric
import numpy as np
import pandas as pd
import matplotlib as plt

In [3]:
skill_outflow_list = pd.read_csv(f"./data/skill_inflow_outflow/skill_outflow_list.csv")
skill_inflow_list = pd.read_csv(f"./data/skill_inflow_outflow/skill_inflow_list.csv")

In [4]:
# skill_inflow_list
skill_outflow_list

,Unnamed: 0,0,1,5,7,11,12,18,20,23,...,24268,24269,24270,24271,24274,24275,24279,24280,24281,time_attr
0,0,3961,3560,2592,2122,1507,1350,1639,1262,1038,...,2,0,0,0,0,0,0,0,0,2017-10
1,1,3242,2979,2010,1713,1265,1112,1364,1066,835,...,0,0,0,0,0,0,0,0,0,2017-11
2,2,3790,3709,2381,2108,1386,1415,1704,1268,1060,...,1,1,0,1,0,0,0,0,0,2017-12
3,3,3615,3481,2352,2009,1371,1319,1497,1139,1039,...,0,0,0,0,0,1,0,0,0,2018-01
4,4,2738,2472,1822,1519,1066,1015,1111,818,791,...,0,0,0,0,0,0,0,0,0,2018-02
5,5,3003,2754,1844,1608,1233,1111,1228,944,841,...,0,0,0,0,0,1,0,0,0,2018-03
6,6,2680,2545,1674,1456,1072,956,1175,891,793,...,0,0,0,0,0,1,0,0,0,2018-04
7,7,2522,2759,1607,1441,952,849,1386,868,608,...,0,1,0,0,0,1,0,1,2,2018-05
8,8,2337,2275,1416,1300,850,905,1065,735,691,...,0,1,0,1,0,1,0,1,0,2018-06
9,9,1977,1882,1215,1001,793,706,842,624,554,...,0,0,0,0,0,1,1,0,1,2018-07


In [25]:
skill_outflow_list

,0,1,2,3,4,5,6,7,8,9,...,24273,24274,24275,24276,24277,24278,24279,24280,24281,time_attr
0,3961,3560,3471,3534,2313,2592,2855,2122,2349,2147,...,0,0,0,1,0,1,0,0,0,2017-10
1,3242,2979,2750,2966,1939,2010,2322,1713,1983,1819,...,0,0,0,1,0,0,0,0,0,2017-11
2,3790,3709,3328,3569,2243,2381,2876,2108,2474,2272,...,0,0,0,0,1,0,0,0,0,2017-12
3,3615,3481,3105,3223,2080,2352,2637,2009,2200,2045,...,0,0,1,0,2,0,0,0,0,2018-01
4,2738,2472,2264,2345,1519,1822,1894,1519,1615,1452,...,0,0,0,0,0,0,0,0,0,2018-02
5,3003,2754,2563,2474,1742,1844,2051,1608,1709,1518,...,0,0,1,0,1,1,0,0,0,2018-03
6,2680,2545,2271,2406,1605,1674,1936,1456,1651,1496,...,0,0,1,0,1,0,0,0,0,2018-04
7,2522,2759,2415,2614,1521,1607,2214,1441,1964,1810,...,0,0,1,0,0,0,0,1,2,2018-05
8,2337,2275,1902,2111,1358,1416,1717,1300,1490,1317,...,0,0,1,0,0,0,0,1,0,2018-06
9,1977,1882,1557,1748,1114,1215,1364,1001,1186,1028,...,0,0,1,0,0,0,1,0,1,2018-07


In [28]:
skill_outflow_list = pd.read_csv(f"./data/skill_inflow_outflow/skill_outflow_list.csv")
skill_inflow_list = pd.read_csv(f"./data/skill_inflow_outflow/skill_inflow_list.csv")
rm_col = []
for col in skill_inflow_list:
    if((skill_inflow_list[col]==0).all() or (skill_outflow_list[col]==0).all()):
        rm_col.append(col)

In [29]:
len(rm_col)


16836

In [ ]:
skill_inflow_list_update = skill_inflow_list.drop(rm_col,axis=1)
skill_inflow_list_update = skill_outflow_list.drop(rm_col,axis=1)


In [2]:
import pykeops
import os
os.environ['CUDA_PATH'] = '/vol/cuda/10.2.89-cudnn7.6.4.38'
pykeops.test_torch_bindings()    # perform the compilation


[KeOps] Warning : There were warnings or errors compiling formula :
<stdin>:1:18: fatal error: cuda.h: No such file or directory
compilation terminated.

[KeOps] Warning : 
    The location of Cuda header files cuda.h and nvrtc.h could not be detected on your system.
    You must determine their location and then define the environment variable CUDA_PATH,
    either before launching Python or using os.environ before importing keops. For example
    if these files are in /vol/cuda/10.2.89-cudnn7.6.4.38/include you can do :
      import os
      os.environ['CUDA_PATH'] = '/vol/cuda/10.2.89-cudnn7.6.4.38'
    
[KeOps] Compiling cuda jit compiler engine ... 
[KeOps] Warning : There were warnings or errors compiling formula :
/home/wchao/anaconda3/envs/skillkg/lib/python3.7/site-packages/keopscore/binders/nvrtc/nvrtc_jit.cpp:5:19: fatal error: nvrtc.h: No such file or directory
 #include <nvrtc.h>
                   ^
compilation terminated.

OK
[pyKeOps] Compiling nvrtc binder for python 

OSError: /home/wchao/.cache/keops2.1.1/build/nvrtc_jit.so: cannot open shared object file: No such file or directory

In [8]:
# import os

import torch
import networkx as nx
from torch_geometric.utils import to_networkx, from_networkx
torch.cuda.is_available()

graph_dir = "./data/graph_list/graph_list/"
demand_graph = nx.read_gpickle(graph_dir+f"myGraph_{0}.gpickle")
demand_graph = from_networkx(demand_graph, group_edge_attrs=["weight"])
demand_graph.edge_index = demand_graph.edge_index[:, (demand_graph.edge_attr>=1e-3).squeeze()]
demand_graph.edge_attr = demand_graph.edge_attr[(demand_graph.edge_attr>=1e-3).squeeze(), :]
supply_graph = nx.read_gpickle(graph_dir+f"supply_cooc_graph.gpickle")
supply_graph = from_networkx(supply_graph, group_edge_attrs=['weight'])
supply_graph.edge_index = supply_graph.edge_index[:, (supply_graph.edge_attr>=1e-3).squeeze()]
supply_graph.edge_index = supply_graph.edge_index+demand_graph.num_nodes
supply_graph.edge_attr = supply_graph.edge_attr[(supply_graph.edge_attr>=1e-3).squeeze(), :]
# demand_graph.update(supply_graph)

In [9]:
for key, val in demand_graph.item():
    demand_graph.update(supply_graph)

ItemsView({'edge_index': tensor([[   0,    0,    0,  ..., 7444, 7444, 7444],
        [   1,    2,    3,  ..., 7071, 7265, 7302]]), 'edge_attr': tensor([[0.0096],
        [0.0094],
        [0.0083],
        ...,
        [0.0003],
        [0.0003],
        [0.0004]]), 'num_nodes': 7446})